# Video Processing in a Sliding Window (Experiment)

This notebook demonstrates reading a video using OpenCV and processing it **a few frames at a time**, keeping a queue of up to `batch_size` frames and processing it. 
Each time we add a new frame, if the queue is full, we drop the oldest frame. This effectively gives us a rolling/sliding window of frames.

A degenerated usage of processing 1 frame per iteration can be achieved by initializing ProcessVideoExperiment with batch_size=1.

In [ ]:
# If you need to install opencv-python, uncomment and run:
# !pip install opencv-python

import numpy as np
import cv2 as cv
from video_batch_processor import VideoBatchProcessor

### Set Video Path and Batch Size
- `video_path`: the path to your supported video file.
- `batch_size`: how many frames to store in the sliding window.
- `video_capture_object`: the object that provides frames to process. 
                          can be camera/file.

In [ ]:
video_path = "Workable Data/Processed/DPH0_Surface_TL.avi"  # Replace with your actual video
batch_size = 5  # Number of frames in the sliding window
video_capture_object = cv.VideoCapture(video_path)

if not video_capture_object.isOpened():
        raise IOError(f"Cannot open video file: {video_path}")


### Setup Hyper-Parameters
- `super_pixel_dimensions`: a tuple of (x_size, y_size)
- `frame_window_size`: the queue size of the moving window of frames.
- `flow_farneback_params`: the params used for the motion detection algorithm.

In [ ]:
super_pixel_dimensions = (2, 2)  # can be modified
frame_window_size = 5
flow_farneback_params = {
    "flow": None,
    "pyr_scale": 0.5,
    "levels": 3,
    "winsize": 15,
    "iterations": 3,
    "poly_n": 5,
    "poly_sigma": 1.2,
    "flags": 0
    }

### Process Function
We'll define a simple function to illustrate what you might do with each batch (window) of frames.
- Here, we just print out how many frames are in the current window.
- You could replace this with custom logic (e.g., saving images, running inference, etc.).

In [3]:
def process_frame_window(frames: np.array, call_index: int):
    # TODO:
    # version 1.0:
    #   call 1 simple processing function to extract data from frame windows.
    # version 2.0:
    #   in this function i will call different processing functions as 
    #   a series of processing units in a way that i can connect output
    #   of 1 processing unit to another.
    #   this will allow the testing of complex analysis ideas.
    print(f'call index: {call_index}, len: {len(frames)}, shape: {frames[0].shape}')
    pass
    

### Main Loop
Now we'll create an instance of `VideoBatchProcessor` and repeatedly call `next_batch()` until we reach the end of the video.
On each iteration, we pass the resulting frames to `process_frame_window()`.

In [4]:
processor = VideoBatchProcessor(video_path, batch_size)
call_index = 0

try:
    # Keep processing frames streamlined
    while True:
        frames_window = processor.next_batch()

        if frames_window is None: 
            break # No more frames

        call_index += 1

        process_frame_window(frames_window, call_index)

except Exception as e:
    print(f"Error during processing: {e}")
finally:
    processor.close()
    print("\nFinished processing the video.")


Finished processing the video.
